# Pallier Cryptosystem

In [1]:
def gen(n):
    pass

def enc(pk, m):
    pass

def dec(sk, pk, m):
    pass

# PLONK

https://eprint.iacr.org/2019/953.pdf

In [2]:
def gen(n):
    pass

def enc(pk, m):
    pass

def dec(sk, pk, m):
    pass